NAME : PRAVEEN S

REG NO : 212222240078

In [ ]:
pip install git+https://github.com/mimoralea/gym-walk#egg=gym-walk

  Cloning https://github.com/mimoralea/gym-walk to /tmp/pip-install-q457bc39/gym-walk_fbcd65ea87814613a79d2770319ef2d7
  Running command git clone --filter=blob:none --quiet https://github.com/mimoralea/gym-walk /tmp/pip-install-q457bc39/gym-walk_fbcd65ea87814613a79d2770319ef2d7
  Resolved https://github.com/mimoralea/gym-walk to commit 5999016267d6de2f5a63307fb00dfd63de319ac1
  Preparing metadata (setup.py) ... done
  Created wheel for gym-walk: filename=gym_walk-0.0.2-py3-none-any.whl size=4058 sha256=51add9e07dce6b0200e7c00724ee77667dc9bd476122e695215b55f602680f4c
  Stored in directory: /tmp/pip-ephem-wheel-cache-ikus4qz5/wheels/24/fe/c4/0cbc7511d29265bad7e28a09311db3f87f0cafba74af54d530
Successfully built gym-walk


In [ ]:
import warnings ; warnings.filterwarnings('ignore')

import gym, gym_walk
import numpy as np

import random
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)
np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123)


In [ ]:
def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k:v for k,v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi(s)
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [ ]:
def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [ ]:
def probability_success(env, pi, goal_state, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        while not done and steps < max_steps:
            state, _, done, h = env.step(pi(state))
            steps += 1
        results.append(state == goal_state)
    return np.sum(results)/len(results)

In [ ]:
def mean_return(env, pi, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        results.append(0.0)
        while not done and steps < max_steps:
            state, reward, done, _ = env.step(pi(state))
            results[-1] += reward
            steps += 1
    return np.mean(results)

In [ ]:
env = gym.make('SlipperyWalkFive-v0')
P = env.env.P
init_state = env.reset()
goal_state = 6
LEFT, RIGHT = range(2)

In [ ]:
P

{0: {0: [(0.5000000000000001, 0, 0.0, True),
   (0.3333333333333333, 0, 0.0, True),
   (0.16666666666666666, 0, 0.0, True)],
  1: [(0.5000000000000001, 0, 0.0, True),
   (0.3333333333333333, 0, 0.0, True),
   (0.16666666666666666, 0, 0.0, True)]},
 1: {0: [(0.5000000000000001, 0, 0.0, True),
   (0.3333333333333333, 1, 0.0, False),
   (0.16666666666666666, 2, 0.0, False)],
  1: [(0.5000000000000001, 2, 0.0, False),
   (0.3333333333333333, 1, 0.0, False),
   (0.16666666666666666, 0, 0.0, True)]},
 2: {0: [(0.5000000000000001, 1, 0.0, False),
   (0.3333333333333333, 2, 0.0, False),
   (0.16666666666666666, 3, 0.0, False)],
  1: [(0.5000000000000001, 3, 0.0, False),
   (0.3333333333333333, 2, 0.0, False),
   (0.16666666666666666, 1, 0.0, False)]},
 3: {0: [(0.5000000000000001, 2, 0.0, False),
   (0.3333333333333333, 3, 0.0, False),
   (0.16666666666666666, 4, 0.0, False)],
  1: [(0.5000000000000001, 4, 0.0, False),
   (0.3333333333333333, 3, 0.0, False),
   (0.16666666666666666, 2, 0.0, Fa

In [ ]:
init_state

3

In [ ]:
state, reward, done, info = env.step(RIGHT)
print("state:{0} - reward:{1} - done:{2} - info:{3}".format(state, reward, done, info))

state:4 - reward:0.0 - done:False - info:{'prob': 0.5000000000000001}


In [ ]:
pi_1 = lambda s: {
    0:LEFT, 1:RIGHT, 2:RIGHT, 3:RIGHT, 4:RIGHT, 5:LEFT, 6:RIGHT
}[s]
print_policy(pi_1, P, action_symbols=('<', '>'), n_cols=7)

Policy:
|           | 01      > | 02      > | 03      > | 04      > | 05      < |           |


In [ ]:
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_1, goal_state=goal_state)*100,
    mean_return(env, pi_1)))

Reaches goal 93.00%. Obtains an average undiscounted return of 0.9300.


In [ ]:
pi_2 = lambda s: {
    0:RIGHT, 1:RIGHT, 2:RIGHT, 3:RIGHT, 4:RIGHT, 5:RIGHT, 6:RIGHT
}[s]

In [ ]:
print_policy(pi_2, P, action_symbols=('<', '>'), n_cols=7)

Policy:
|           | 01      > | 02      > | 03      > | 04      > | 05      > |           |


In [ ]:
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_2, goal_state=goal_state)*100,
    mean_return(env, pi_2)))

Reaches goal 97.00%. Obtains an average undiscounted return of 0.9700.


In [ ]:
print('First policy')
print_policy(pi_1, P, action_symbols=('<', '>'), n_cols=7)
print('Second policy')
print_policy(pi_2, P, action_symbols=('<', '>'), n_cols=7)
print("\n")
print("The Second Policy is Better")

First policy
Policy:
|           | 01      > | 02      > | 03      > | 04      > | 05      < |           |
Second policy
Policy:
|           | 01      > | 02      > | 03      > | 04      > | 05      > |           |


The Second Policy is Better


In [ ]:
def policy_evaluation(pi,P,gamma=1.0,theta=1e-10):
  prev_V=np.zeros(len(P))
  while True:
    V=np.zeros(len(P))
    for s in range(len(P)):
      for prob,next_state,reward,done in P[s][pi(s)]:
        V[s]+=prob*(reward+gamma*prev_V[next_state]*(not done))
    if np.max(np.abs(V-prev_V)):
      break
    prev_V=V.copy()
  return V

In [ ]:
V1 = policy_evaluation(pi_1, P)
print_state_value_function(V1, P, n_cols=7, prec=5)

State-value function:
|           | 01    0.0 | 02    0.0 | 03    0.0 | 04    0.0 | 05 0.16667 |           |


In [ ]:
V2 = policy_evaluation(pi_2, P)
print_state_value_function(V2, P, n_cols=7, prec=5)

State-value function:
|           | 01    0.0 | 02    0.0 | 03    0.0 | 04    0.0 | 05    0.5 |           |


In [ ]:
print("The State One of First policy is 01 0.08333  and The State One of Second policy is 01 0.17308.")
print("So the Second Policy is better Policy")

The State One of First policy is 01 0.08333  and The State One of Second policy is 01 0.17308.
So the Second Policy is better Policy


In [ ]:
V1

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.16666667, 0.        ])

In [ ]:
print_state_value_function(V1, P, n_cols=7, prec=5)

State-value function:
|           | 01    0.0 | 02    0.0 | 03    0.0 | 04    0.0 | 05 0.16667 |           |


In [ ]:
V2

array([0. , 0. , 0. , 0. , 0. , 0.5, 0. ])

In [ ]:
print_state_value_function(V2, P, n_cols=7, prec=5)

State-value function:
|           | 01    0.0 | 02    0.0 | 03    0.0 | 04    0.0 | 05    0.5 |           |


In [ ]:
V1>=V2

array([ True,  True,  True,  True,  True, False,  True])

In [ ]:
if(np.sum(V1>=V2)==7):
  print("The first policy is the better policy")
elif(np.sum(V2>=V1)==7):
  print("The second policy is the better policy")
else:
  print("Both policies have their merits.")

The second policy is the better policy
